In [12]:
import requests 
import urllib
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [15]:
root = 


soup = BeautifulSoup(page.content, 'html.parser')

In [28]:
page.raise_for_status()

In [67]:
entries = page.json()['items']


In [53]:
[0]['stationReference']

'1029TH'

In [232]:
import numpy as np
import datetime as dt
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [307]:
class MeasurementTool():
    def __init__(self,):
        self.root='https://environment.data.gov.uk/flood-monitoring'
        self.page_url = self.root + '/id/stations/'
        self.page = requests.get(self.page_url)
        self.entries = self.page.json()['items']
        self.stations = self.create_stations_list()
        self.data = defaultdict(lambda: {"timestamps": [], "values": []})
    
    def retrieve_measurement(self,station_id):
        station_url = self.page_url + station_id
        latest_possible_reading_time = self.get_rounded_timestamp()
        last_24_hrs = self.subtract_24_hours(latest_possible_reading_time)
        station_page = requests.get(station_url + "/readings?since=" + last_24_hrs)
        station_data = station_page.json()["items"]

        for item in station_data:
            measure_name = "-".join(item["measure"].split("/")[-1].split("-")[1:])  # Extract measure name
            timestamp = item["dateTime"]
            value = item["value"]
            self.data[measure_name]["timestamps"].append(timestamp)
            self.data[measure_name]["values"].append(value)

    def get_rounded_timestamp(self):
        """Returns the current timestamp rounded down to the nearest 15-minute increment."""
        now = dt.datetime.now(dt.timezone.utc)
        rounded_minutes = now.minute - (now.minute % 15)  # Round down to nearest 15-minute mark
        rounded_time = now.replace(minute=rounded_minutes, second=0, microsecond=0)
        return rounded_time.strftime("%Y-%m-%dT%H:%M:%SZ")

    def subtract_24_hours(self,timestamp: str) -> str:
        """Subtracts 24 hours from a given timestamp and returns it in the same format."""
        datetime = dt.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        new_dt = datetime - dt.timedelta(hours=24)
        return new_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
         
    def create_stations_list(self):
        stations = []
        station_ids = []
        for entry in self.entries: 
            stations.append(entry['@id'])
            station_ids.append(entry['stationReference'])
        return station_ids

    def plot_measure(self,df):

        # Convert timestamp to datetime
        df['timestamps'] = pd.to_datetime(df['timestamps'])

        # Plot
        plt.figure(figsize=(10, 5))


        # Formatting
        plt.xlabel("Timestamp")
        plt.ylabel("Value")
        plt.title("Time Series Data")
        plt.xticks(rotation=45)  # Rotate x-axis labels for readability
        plt.grid()
        plt.plot(df['timestamps'], df['values'], marker='o', linestyle='-')
    

    def create_widgets(self):
        global df
        df = None
        # Output widget to display results
        output = widgets.Output()

        # First dropdown for station selection
        station_selector = widgets.Dropdown(
            options=[" "] + self.stations,  # List of station IDs with blank
            description="Station:",
            style={'description_width': 'initial'}
        )

        # Second dropdown for measure selection (Initially empty)
        measure_selector = widgets.Dropdown(
            options=[],  # Will be updated dynamically
            description="Measure:",
            style={'description_width': 'initial'}
        )

        # Button to retrieve data
        fetch_button = widgets.Button(description="Retrieve Data")

        # Function to update measure selector based on selected station
        def update_measure_selector(change):
            station_id = change.new  # Get newly selected station
            if station_id:
                output.clear_output()
                print(f"Fetching data for {station_id}...")

                # Fetch data for the station (Replace this with your function call)
                self.retrieve_measurement(station_id)  # Your function
                # Update measure dropdown with available measures
                measure_selector.options = [" "] + list(self.data.keys())

        # Attach the function to station dropdown
        station_selector.observe(update_measure_selector, names="value")

        # Function to handle data retrieval
        def on_data_button_click(b):
            with output:
                output.clear_output()
                station_id = station_selector.value
                measure = measure_selector.value

                if not station_id:
                    print("Please select a station.")
                    return
                
                if not measure:
                    print("Please select a measure.")
                    return

                print(f"Fetching data for {station_id}, measure: {measure}...")
                
                # Fetch and display data (Replace this with your actual function)
                df = self.data[measure] if measure in self.data else None
                print(df)
                if df is None:
                    print("No data available for the selected measure.")
                else:
                    _ = interact(self.plot_measure(df))

        # Attach button to function
        fetch_button.on_click(on_data_button_click)

        # Display widgets
        display(station_selector, measure_selector, fetch_button, output)


In [308]:
tool = MeasurementTool()
tool.create_widgets()

Dropdown(description='Station:', options=(' ', '1029TH', 'E2043', '52119', 'E21136', '2067', '48143', '720215'…

Dropdown(description='Measure:', options=(), style=DescriptionStyle(description_width='initial'), value=None)

Button(description='Retrieve Data', style=ButtonStyle())

Output()

Fetching data for 1029TH...


In [198]:
data = tool.retrieve_measurement('1029TH')